In [3]:
import tensorflow as tf
import numpy as np

In [14]:
def IOU(bndbox_0, bndbox_1, S, B):
    #  bndbox : n, S, S, 1, 5 / n, S, S, B, 5
    bndbox_0 = tf.tile(bndbox_0, (1, 1, 1, B, 1))
    bndbox = tf.stack([bndbox_0, bndbox_1], axis=-1)  # n, S, S, B, 5, 2
    print(bndbox)
    left_top = tf.reduce_max(bndbox[..., :2, :] / S - tf.square(bndbox[..., 2:4, :]) / 2, axis=-1)  # n, S, S, B, 2
    right_bot = tf.reduce_min(bndbox[..., :2, :] / S + tf.square(bndbox[..., 2:4, :]) / 2, axis=-1)
    small_wh = tf.reduce_min(tf.square(bndbox[..., 2:4, :]), axis=-1)
    intersection_wh = tf.clip_by_value(right_bot - left_top, 0., small_wh)  # n, S, S, B, 2
    intersection = tf.reduce_prod(intersection_wh, axis=-1)  # n, S, S, B
    union = tf.reduce_sum(tf.reduce_prod(tf.square(bndbox[..., 2:4, :]), axis=-2), axis=-1) - intersection
    return intersection / (union + 1e-4)

In [37]:
b_1 = np.array([511, 41, 577, 76])
b_2 = np.array([[554, 69, 620, 104], [544, 59, 610, 94]])

b_1 = np.array([(b_1[2]+b_1[0])/2, (b_1[3]+b_1[1])/2, np.sqrt(b_1[2]-b_1[0]), np.sqrt(b_1[3]-b_1[1]), 0]).reshape(1, 1, 1, 1, 5)
b_2 = np.array([(b_2[:, 2]+b_2[:, 0])/2, (b_2[:, 3]+b_2[:, 1])/2, np.sqrt(b_2[:, 2]-b_2[:, 0]), np.sqrt(b_2[:, 3]-b_2[:, 1]), [0, 0]]).reshape(1, 1, 1, 5, 2).transpose(0, 1, 2, 4, 3)
print(b_2)

[[[[[587.          86.5          8.1240384    5.91607978   0.        ]
    [577.          76.5          8.1240384    5.91607978   0.        ]]]]]


In [38]:
iou = IOU(b_1, b_2, 1, 2)

tf.Tensor(
[[[[[[544.         587.        ]
     [ 58.5         86.5       ]
     [  8.1240384    8.1240384 ]
     [  5.91607978   5.91607978]
     [  0.           0.        ]]

    [[544.         577.        ]
     [ 58.5         76.5       ]
     [  8.1240384    8.1240384 ]
     [  5.91607978   5.91607978]
     [  0.           0.        ]]]]]], shape=(1, 1, 1, 2, 5, 2), dtype=float64)


In [39]:
tf.one_hot(tf.argmax(iou, axis=-1), depth=2, axis=-1)

<tf.Tensor: shape=(1, 1, 1, 2), dtype=float32, numpy=array([[[[0., 1.]]]], dtype=float32)>

In [1]:
!export PYTHON_PATH=$PYTHON_PATH:/opt/project/sandbox_1/

In [2]:
from yolo_config import CFG
from model import YoloV1
from loss import YoloV1Loss
from data import YoloData

from config import dict2namedtuple
from data_load.data_loader import DataLoader
from optimizer import get_optimizer
from learning_rate import get_learning_rate

import time

In [3]:
params = dict2namedtuple(CFG)

In [4]:
data_loader = DataLoader(params)
yolo_data = YoloData(params)

In [5]:
ds_0 = data_loader.set_ds(training=True)

In [6]:
gen = yolo_data.train_batch_generator(ds_0, 16, 8)

In [7]:
t_1 = time.time()
for i, (x, y) in enumerate(gen):
    if i >20:
        break
    print(x.shape, y.shape)
print(time.time()-t_1)

(16, 448, 448, 3) (16, 7, 7, 30)
(16, 448, 448, 3) (16, 7, 7, 30)
(16, 448, 448, 3) (16, 7, 7, 30)
(16, 448, 448, 3) (16, 7, 7, 30)
(16, 448, 448, 3) (16, 7, 7, 30)
(16, 448, 448, 3) (16, 7, 7, 30)
(16, 448, 448, 3) (16, 7, 7, 30)
(16, 448, 448, 3) (16, 7, 7, 30)
(16, 448, 448, 3) (16, 7, 7, 30)
(16, 448, 448, 3) (16, 7, 7, 30)
(16, 448, 448, 3) (16, 7, 7, 30)
(16, 448, 448, 3) (16, 7, 7, 30)
(16, 448, 448, 3) (16, 7, 7, 30)
(16, 448, 448, 3) (16, 7, 7, 30)
(16, 448, 448, 3) (16, 7, 7, 30)
(16, 448, 448, 3) (16, 7, 7, 30)
(16, 448, 448, 3) (16, 7, 7, 30)
(16, 448, 448, 3) (16, 7, 7, 30)
(16, 448, 448, 3) (16, 7, 7, 30)
(16, 448, 448, 3) (16, 7, 7, 30)
(16, 448, 448, 3) (16, 7, 7, 30)
18.585267066955566


In [8]:
ds_1 = data_loader.set_ds(yolo_data.map_fn_train, training=True)

In [9]:
t_1 = time.time()
for i, (x, y) in enumerate(ds_1):
    if i > 20:
        break
    print(x.shape, y.shape)
print(time.time()-t_1)

(8, 448, 448, 3) (8, 7, 7, 30)
(8, 448, 448, 3) (8, 7, 7, 30)
(8, 448, 448, 3) (8, 7, 7, 30)
(8, 448, 448, 3) (8, 7, 7, 30)
(8, 448, 448, 3) (8, 7, 7, 30)
(8, 448, 448, 3) (8, 7, 7, 30)
(8, 448, 448, 3) (8, 7, 7, 30)
(8, 448, 448, 3) (8, 7, 7, 30)
(8, 448, 448, 3) (8, 7, 7, 30)
(8, 448, 448, 3) (8, 7, 7, 30)
(8, 448, 448, 3) (8, 7, 7, 30)
(8, 448, 448, 3) (8, 7, 7, 30)
(8, 448, 448, 3) (8, 7, 7, 30)
(8, 448, 448, 3) (8, 7, 7, 30)
(8, 448, 448, 3) (8, 7, 7, 30)
(8, 448, 448, 3) (8, 7, 7, 30)
(8, 448, 448, 3) (8, 7, 7, 30)
(8, 448, 448, 3) (8, 7, 7, 30)
(8, 448, 448, 3) (8, 7, 7, 30)
(8, 448, 448, 3) (8, 7, 7, 30)
(8, 448, 448, 3) (8, 7, 7, 30)
5.976024866104126
